<p style="font-weight:bold;"> <span style="font-size: 36px"> IFRS17 Calculation Engine </span> </p>

In [ ]:
#!import "//ifrs17/dev/CalculationEngine"

In [ ]:
#r "nuget:Systemorph.Charting,1.5.5"

# TMP report configurations

In [ ]:
// TODO: this must be improved and included in the api
const string ArrayFormatter = "value.map(v => new Intl.NumberFormat([], { maximumFractionDigits: 8 }).format(v)).join(' , ')";

In [ ]:
// TODO: the default must include this 
public static ReportBuilder<YieldCurve,YieldCurve,YieldCurve> WithGridOptionsForYieldCurves (this ReportBuilder<YieldCurve,YieldCurve,YieldCurve> reportBuilder, int reportHeight = 200)
{
    return reportBuilder.WithGridOptions(o => o.WithColumns(c => c.Modify("Values", x => x with { ValueFormatter = ArrayFormatter, CellStyle = new { TextAlign = "left" } } ))
                                         with { Height = reportHeight, TreeData = false } );  // AutoGroupColumnDef = null (what is this?)
}

In [ ]:
using System.Collections.Immutable;
// TODO: The type or namespace name 'ToImmutableList' does not exist in the namespace 'System.Collections.Immutable
//System.Collections.Immutable.ToImmutableList(r.RowGroup.Coordinates.Where(c => c != "NullGroup")) 

// TODO: discuss default here too, especially the trick to remove empty row
public static ReportBuilder<RawVariable,RawVariable,RawVariable> WithGridOptionsForCashflows (this ReportBuilder<RawVariable,RawVariable,RawVariable> reportBuilder, int reportHeight = 450)
{
    return reportBuilder.GroupColumnsBy(x => x.DataNode)
                        //.GroupColumnsBy(x => x.CalculationType)
                        .GroupRowsBy(x => x.AmountType)
                        .GroupRowsBy(x => x.AocType)
                        .WithGridOptions(o => o.WithRows(r => r.Where(r => !(r.RowGroup.Coordinates.Last() == "NullGroup"))
                                                               .Select(r => r with { RowGroup = r.RowGroup with { Coordinates = r.RowGroup.Coordinates.Where(c => c != "NullGroup").ToImmutableList() } }).ToList())
                                               .WithColumns(c => c.Modify("Values", x => x with { ValueFormatter = ArrayFormatter, CellStyle = new { TextAlign = "left" } } ))
                                         with { Height = reportHeight, GroupDefaultExpanded = 2 } );
}

In [ ]:
public static DataCubeReportBuilder<IDataCube<RawVariable>, RawVariable, RawVariable ,RawVariable> WithGridOptionsForCashflows 
(this DataCubeReportBuilder<IDataCube<RawVariable>, RawVariable, RawVariable ,RawVariable> reportBuilder, int reportHeight = 450)
{
    return reportBuilder.SliceColumnsBy(nameof(GroupOfContract))//,nameof(CalculationType))
                        .SliceRowsBy(nameof(AmountType),nameof(AocType))
                        .WithGridOptions(o => o.WithRows(r => r.Where(r => !(r.RowGroup.Coordinates.Last() == "NullGroup"))
                                                               .Select(r => r with { RowGroup = r.RowGroup with { Coordinates = r.RowGroup.Coordinates.Where(c => c != "NullGroup").ToImmutableList() } }).ToList())
                                               .WithColumns(c => c.Modify("Values", x => x with { ValueFormatter = ArrayFormatter, CellStyle = new { TextAlign = "left" } } ))
                                         with { Height = reportHeight, GroupDefaultExpanded = 2 } );
}

# TMP Report Chart

In [ ]:
static Systemorph.Vertex.Charting.Builders.ChartBuilderVariable chart;
chart = (Systemorph.Vertex.Charting.Builders.ChartBuilderVariable)ChartBuilder;

public static YieldCurve[] ForObject
(this Systemorph.Vertex.Pivot.Builder.Interfaces.IPivotFactory report, YieldCurve[] data) 
    => data;

public static Systemorph.Vertex.DataCubes.DataCube<RawVariable> ForDatacube
(this Systemorph.Vertex.Pivot.Builder.Interfaces.IPivotFactory report, Systemorph.Vertex.DataCubes.Api.IDataCube<RawVariable> data) 
    => (Systemorph.Vertex.DataCubes.DataCube<RawVariable>)data;

public static Systemorph.Vertex.Charting.Builders.ChartBuilders.LineChartBuilder WithOptionsForEconomicRates 
(this YieldCurve[] ycs)
{
    var res = chart.Line();
    var max = -100.0;
    var min = +100.0;
    foreach(var yc in ycs) 
    {
        var interest = (from x in yc.Values select Math.Pow((1 + x),( +1.0 / 12.0 ))).ToArray();
        //var discount = (from x in yc.Values select (1 + x)^( -1.0 / 12.0 )).ToArray();
        if(interest.Max() > max) max = interest.Max();
        //if(discount.Max() > max) max = discount.Max();
        if(interest.Min() < min) min = interest.Min();
        //if(discount.Min() < min) min = discount.Min();
        res.WithDataSet( o => o.WithData(interest).WithLabel( yc.Currency) );
        //.WithDataArray( discount , o => o.WithLabel("Discount " + yc.Currency) );
    }
    return res.WithOptions(o => o.WithYAxisMin(Math.Truncate((min-0.001) * 1000) / 1000))
              .WithOptions(o => o.WithYAxisMax(Math.Truncate((max+0.001) * 1000) / 1000))
              .WithTitle("Discount Factor")
              .WithLegend();
}

public static Systemorph.Vertex.Charting.Builders.ChartBuilders.BarChartBuilder WithOptionsForNominalCashFlows
(this Systemorph.Vertex.DataCubes.DataCube<RawVariable> cfs)
{
    return chart.Bar().WithTitle("Nominal Cashflows", o => o.WithFontSize(20))
                      .WithDataSet(o => o.WithData(cfs.First(x => x.AmountType == "PR" && x.AocType == "BOP") .Values).WithLabel("Premiums BOP"))
                      .WithDataSet(o => o.WithData(cfs.First(x => x.AmountType == "PR" && x.AocType == "AU")  .Values).WithLabel("Premiums AU"))
                      .WithDataSet(o => o.WithData(cfs.First(x => x.AmountType == "PR" && x.AocType == "EV")  .Values).WithLabel("Premiums EV"))
                      .WithDataSet(o => o.WithData(cfs.First(x => x.AmountType == "PR" && x.AocType == "CL")  .Values).WithLabel("Premiums CL"))
                      .WithDataSet(o => o.WithData(cfs.First(x => x.AmountType == "CL" && x.AocType == "BOP") .Values).WithLabel("Claims BOP"))
                      .WithDataSet(o => o.WithData(cfs.First(x => x.AmountType == "CL" && x.AocType == "EV")  .Values).WithLabel("Claims EV"))
                      .WithDataSet(o => o.WithData(cfs.First(x => x.AmountType == "CL" && x.AocType == "CL")  .Values).WithLabel("Claims CL"))
                      .WithLegend();
}

public static Systemorph.Vertex.Charting.ChartPresenterData ToLineChart
(this Systemorph.Vertex.Charting.Builders.ChartBuilders.LineChartBuilder charting) 
    => charting.ToChart();

public static Systemorph.Vertex.Charting.ChartPresenterData ToBarChart
(this Systemorph.Vertex.Charting.Builders.ChartBuilders.BarChartBuilder charting) 
    => charting.ToChart();